In [1]:
import whisper
from datasets import load_dataset, Dataset, Audio

/Users/maltegenschow/.pyenv/versions/speech/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Creating an audio dataset from .wav files

In [2]:
from glob import glob
files = glob('../voice_cloning/voices/*.wav')

dataset = Dataset.from_dict({
    'name':[elem.split('/')[-1].split('.')[0] for elem in files],
    'path':files,
    'audio':files}).cast_column("audio", Audio(sampling_rate=16000))
dataset[0]

{'name': 'musk',
 'path': '../voice_cloning/voices/musk.wav',
 'audio': {'path': '../voice_cloning/voices/musk.wav',
  'array': array([-0.0107157 , -0.0207513 , -0.01781394, ..., -0.02346013,
          0.08692658,  0.        ]),
  'sampling_rate': 16000}}

In [5]:
id = 4
display(dataset[id])

model = whisper.load_model("base")
model.transcribe(dataset[id]['audio']['array'].astype('float32'))['text']

{'name': 'trump',
 'path': '../voice_cloning/voices/trump.wav',
 'audio': {'path': '../voice_cloning/voices/trump.wav',
  'array': array([ 0.14260465,  0.2145554 ,  0.15499626, ..., -0.12529853,
         -0.21823871,  0.        ]),
  'sampling_rate': 16000}}

" election. The election was rigged. There's no question about that. There's so much proof on it. Even if you go to the more modern-day proof with the, they go at Twitter files, FBI and Twitter, or you take a look at the Amazon stuff or the Google stuff or you take a look at 2000."

In [ ]:
model.transcribe("audio/afjiv.wav")['text']

### Pipeline

1. Speaker diarization
2. Audio splitting based on speaker diarization
3. Chunk transcription

In [ ]:
# Run Speaker diarization
from pyannote.audio import Pipeline
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-2.1")

diarization = pipeline("audio/afjiv.wav", num_speakers = 2)
for turn, _, speaker in diarization.itertracks(yield_label=True,):
    print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")

In [ ]:
# Split the audion file into speaker chunks
from pydub import AudioSegment

audio = AudioSegment.from_wav('audio/afjiv.wav')

chunks = []
for turn, _, speaker in diarization.itertracks(yield_label=True):
    start_ms = int(turn.start * 1000)
    end_ms = int(turn.end * 1000)
    chunk_audio = audio[start_ms:end_ms]
    chunk_audio.export(f"{speaker}_{start_ms}_{end_ms}.wav", format="wav")
    chunks.append({
        'file': f"splitted_speaker_files/{speaker}_{start_ms}_{end_ms}.wav",
        'speaker': speaker,
        'start': turn.start,
        'end': turn.end
    })
chunks

In [ ]:
# Do the actual transcribing using whisper
import whisper
model = whisper.load_model("base")

final_transcript = []

for chunk in chunks:
    result = model.transcribe(chunk['file'])
    final_transcript.append({
        'speaker': chunk['speaker'],
        'start': chunk['start'],
        'end': chunk['end'],
        'text': result['text']
    })

In [ ]:
for entry in final_transcript:
    print(f"[{entry['start']:.2f}–{entry['end']:.2f}] {entry['speaker']}: {entry['text']}")


### Using whisper-x diarization and alignment

In [54]:
#! pip install git+https://github.com/m-bain/whisperX.git

In [23]:
import torch

In [ ]:
# Load one example
ds = load_dataset("talkbank/callhome", "deu", streaming=True)
example = next(iter(ds['data']))

import soundfile as sf
sf.write('phone_call.wav', example['audio']['array'], samplerate=example['audio']['sampling_rate'])

In [52]:
audio_path = 'audio/phone_call.wav'

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [45]:
import whisper

model_name = "base"  
model = whisper.load_model(model_name)
script = model.transcribe(audio_path)
script['text']

/Users/maltegenschow/.pyenv/versions/speech/lib/python3.11/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, m

" Ja? Ich bin gerade auf dem Weg nach Hause, eigentlich? Oh, okay. Ich wollte gerade losfahren, aber ich habe hier nicht hängen, gerade noch vom Fernseher. Von daher hat er noch ein bisschen Zeit. Sicher? Ja. Was willst alle wissen? Wir haben Schultenfest in Barloch. Oh, in Wo? In Barloch. Barloch? Ja, wo ich meine Heimat statt. Okay, ich wusste nicht, wie die heißt. Die halb doch vier. Vier sind jetzt meine Wahlheimat. Vier sind Bospheim. Ja? Da ist jetzt auch angerufen. Also genau. Da wohnt's doch jetzt? Ja, da wohne ich jetzt auch. Ja. Aber ähm... Ich komme hier aus Barloch. Und ich bin nicht immer noch als meine Heimat statt. Okay. Auf jeden Fall ist der Schultenfest und da muss ich jetzt dieses Wochenende noch hin. Ah. Im Mund auch noch holopp genommen. Echt? Ja. Wir treffen fest. Ja, ein Tag. Ist schlecht. Ja, und dann... Und ein Dienstach muss man direkt von das Firma nach Peine hin. Ja. Ich hoffe, dass ich das schon wieder nicht kam mit. Könnte sonst ein bisschen hart werden. M

In [48]:
from whisperx.diarize import DiarizationPipeline

diarization_pipeline = DiarizationPipeline()
diarized = diarization_pipeline(audio_path)
diarized

/Users/maltegenschow/.pyenv/versions/speech/lib/python3.11/site-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1808.)
  std = sequences.std(dim=-1, correction=1)


,segment,label,speaker,start,end
0,[ 00:00:00.030 --> 00:00:02.950],A,SPEAKER_03,0.030969,2.950344
1,[ 00:00:03.186 --> 00:00:04.097],B,SPEAKER_01,3.186594,4.097844
2,[ 00:00:04.097 --> 00:00:05.734],C,SPEAKER_03,4.097844,5.734719
3,[ 00:00:05.835 --> 00:00:08.924],D,SPEAKER_03,5.835969,8.924094
4,[ 00:00:09.075 --> 00:00:11.894],E,SPEAKER_03,9.075969,11.894094
...,...,...,...,...,...
377,[ 00:09:57.945 --> 00:09:59.076],NN,SPEAKER_03,597.945969,599.076594
378,[ 00:09:59.059 --> 00:09:59.650],NO,SPEAKER_00,599.059719,599.650344
379,[ 00:09:59.869 --> 00:10:00.932],NP,SPEAKER_00,599.869719,600.932844
380,[ 00:09:59.970 --> 00:10:00.173],NQ,SPEAKER_03,599.970969,600.173469


In [49]:
from whisperx import load_align_model, align
from whisperx.diarize import assign_word_speakers

In [53]:
# Align Script
model_a, metadata = load_align_model(language_code=script["language"], device = device)
script_aligned = align(script["segments"], model_a, metadata, audio_path, device)

# Align Speakers
result_segments, word_seg = list(assign_word_speakers(
    diarized, script_aligned
).values())
transcribed = []
for result_segment in result_segments:
    transcribed.append(
        {
            "start": result_segment["start"],
            "end": result_segment["end"],
            "text": result_segment["text"],
            "speaker": result_segment["speaker"],
        }
    )

KeyError: 'speaker'

In [56]:
for start, end, text, speaker in [i.values() for i in transcribed]:
    print(start, end, speaker, text)

0.0 1.021 SPEAKER_03  Ja?
1.0 3.02 SPEAKER_03  Ich bin gerade auf dem Weg nach Hause, eigentlich?
3.0 4.021 SPEAKER_01  Oh, okay.
4.0 8.02 SPEAKER_03  Ich wollte gerade losfahren, aber ich habe hier nicht hängen, gerade noch vom Fernseher.
8.0 12.02 SPEAKER_03  Von daher hat er noch ein bisschen Zeit.
12.0 13.021 SPEAKER_01  Sicher?
13.0 14.021 SPEAKER_01  Ja.
14.0 17.02 SPEAKER_03  Was willst alle wissen?
17.0 19.02 SPEAKER_03  Wir haben Schultenfest in Barloch.
19.0 20.021 SPEAKER_03  Oh, in Wo?
20.0 21.021 SPEAKER_03  In Barloch.
21.0 22.021 SPEAKER_00  Barloch?
22.0 25.02 SPEAKER_03  Ja, wo ich meine Heimat statt.
25.0 28.02 SPEAKER_00  Okay, ich wusste nicht, wie die heißt.
28.0 30.02 SPEAKER_00  Die halb doch vier.
30.0 32.02 SPEAKER_03  Vier sind jetzt meine Wahlheimat.
32.0 34.02 SPEAKER_03  Vier sind Bospheim.
34.0 35.021 SPEAKER_03  Ja?
35.0 37.02 SPEAKER_03  Da ist jetzt auch angerufen.
37.0 38.021 SPEAKER_00  Also genau.
38.0 40.02 SPEAKER_00  Da wohnt's doch jetzt?
40.0 41

### WhisperX from official docs

In [59]:
#!pip install whisperx

In [65]:
# Setup
import whisperx
import gc 

device = "cpu" 
audio_file = "audio/phone_call.wav"
batch_size = 16 # reduce if low on GPU mem
compute_type = "float32" # change to "int8" if low on GPU mem (may reduce accuracy)

In [67]:
# 1. Transcribe with original whisper (batched)
model = whisperx.load_model("base", device, compute_type=compute_type)

audio = whisperx.load_audio(audio_file)
result = model.transcribe(audio, batch_size=batch_size)
print(result["segments"]) # before alignment

Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../.pyenv/versions/speech/lib/python3.11/site-packages/whisperx/assets/pytorch_model.bin`


No language specified, language will be first be detected for each audio file (increases inference time).
>>Performing voice activity detection using Pyannote...
Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.4.1. Bad things might happen unless you revert torch to 1.x.
Detected language: de (0.96) in first 30s of audio...
[{'text': ' Ja? Ja, ich bin gerade auf dem Weg nach Hause, eigentlich? Oh, okay. Ich wollte gerade losfahren, aber ich habe hier nicht hängen, gerade noch vom Fernseher. Von daher hatte es noch ein bisschen Zeit. Sicher? Ja. Was wollt es alles wissen? Wir haben Schutzensfest in Barloch. Oh, in Wo? In Barloch. Barloch? Ja, wo ich meine Heimat statt. Okay, ich wusste nicht, wie die heißt.', 'start': 0.031, 'end': 28.195}, {'text': ' Die halb doch vierze vierze ist jetzt meine Wahlheimat vierze Boothheim ja da ist jetzt auch anrufen also ge

In [68]:
# 2. Align whisper output
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

print(result["segments"]) # after alignment

[{'start': 0.031, 'end': 1.173, 'text': ' Ja?', 'words': [{'word': 'Ja?', 'start': 0.031, 'end': 1.173, 'score': 0.299}]}, {'start': 1.193, 'end': 4.358, 'text': 'Ja, ich bin gerade auf dem Weg nach Hause, eigentlich?', 'words': [{'word': 'Ja,', 'start': 1.193, 'end': 1.253, 'score': 0.152}, {'word': 'ich', 'start': 1.273, 'end': 1.373, 'score': 0.921}, {'word': 'bin', 'start': 1.393, 'end': 1.513, 'score': 0.951}, {'word': 'gerade', 'start': 1.553, 'end': 1.734, 'score': 0.702}, {'word': 'auf', 'start': 1.754, 'end': 1.834, 'score': 0.867}, {'word': 'dem', 'start': 1.854, 'end': 1.934, 'score': 0.587}, {'word': 'Weg', 'start': 1.954, 'end': 2.134, 'score': 0.722}, {'word': 'nach', 'start': 2.174, 'end': 2.275, 'score': 0.655}, {'word': 'Hause,', 'start': 2.315, 'end': 2.575, 'score': 0.482}, {'word': 'eigentlich?', 'start': 2.595, 'end': 4.358, 'score': 0.86}]}, {'start': 4.378, 'end': 4.558, 'text': 'Oh, okay.', 'words': [{'word': 'Oh,', 'start': 4.378, 'end': 4.438, 'score': 0.038},

In [69]:
# 3. Assign speaker labels
diarize_model = whisperx.DiarizationPipeline(device=device)

# add min/max number of speakers if known
diarize_segments = diarize_model(audio, num_speakers=2)


/Users/maltegenschow/.pyenv/versions/speech/lib/python3.11/site-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1808.)
  std = sequences.std(dim=-1, correction=1)


In [70]:
result = whisperx.assign_word_speakers(diarize_segments, result)
print(diarize_segments)
print(result["segments"]) # segments are now assigned speaker IDs

                               segment label     speaker       start  \
0    [ 00:00:00.030 -->  00:00:02.950]     A  SPEAKER_01    0.030969   
1    [ 00:00:03.186 -->  00:00:04.097]     B  SPEAKER_00    3.186594   
2    [ 00:00:04.097 -->  00:00:05.734]     C  SPEAKER_01    4.097844   
3    [ 00:00:05.835 -->  00:00:08.924]     D  SPEAKER_01    5.835969   
4    [ 00:00:09.075 -->  00:00:11.894]     E  SPEAKER_01    9.075969   
..                                 ...   ...         ...         ...   
351  [ 00:09:57.945 -->  00:09:59.076]    MN  SPEAKER_01  597.945969   
352  [ 00:09:59.059 -->  00:09:59.650]    MO  SPEAKER_00  599.059719   
353  [ 00:09:59.869 -->  00:10:00.932]    MP  SPEAKER_00  599.869719   
354  [ 00:09:59.970 -->  00:10:00.173]    MQ  SPEAKER_01  599.970969   
355  [ 00:10:01.557 -->  00:10:01.962]    MR  SPEAKER_01  601.557219   

            end  intersection       union  
0      2.950344   -598.944656  601.884031  
1      4.097844   -597.797156  598.728406  
2  

In [75]:
for segment in result["segments"]:
    start = f"{segment['start']:.2f}".rjust(6)
    end = f"{segment['end']:.2f}".ljust(6)
    speaker = segment.get('speaker', 'Unknown').ljust(10)
    text = segment['text'].strip()

    print(f"{start} → {end} | {speaker:<12} | {text}")


  0.03 → 1.17   | SPEAKER_01   | Ja?
  1.19 → 4.36   | SPEAKER_01   | Ja, ich bin gerade auf dem Weg nach Hause, eigentlich?
  4.38 → 4.56   | SPEAKER_01   | Oh, okay.
  4.58 → 8.14   | SPEAKER_01   | Ich wollte gerade losfahren, aber ich habe hier nicht hängen, gerade noch vom Fernseher.
  9.45 → 12.55  | SPEAKER_01   | Von daher hatte es noch ein bisschen Zeit.
 12.57 → 16.46  | SPEAKER_01   | Sicher?
 16.48 → 16.54  | SPEAKER_01   | Ja.
 16.56 → 17.58  | SPEAKER_01   | Was wollt es alles wissen?
 17.62 → 19.74  | SPEAKER_01   | Wir haben Schutzensfest in Barloch.
 19.76 → 20.82  | SPEAKER_01   | Oh, in Wo?
 20.84 → 21.75  | SPEAKER_01   | In Barloch.
 21.77 → 22.99  | SPEAKER_01   | Barloch?
 23.01 → 26.23  | SPEAKER_01   | Ja, wo ich meine Heimat statt.
 26.25 → 28.21  | SPEAKER_00   | Okay, ich wusste nicht, wie die heißt.
 28.40 → 48.23  | SPEAKER_01   | Die halb doch vierze vierze ist jetzt meine Wahlheimat vierze Boothheim ja da ist jetzt auch anrufen also genau da wohnst du do